In [1]:
from datasets import load_dataset
import pandas as pd
from string import ascii_uppercase

In [2]:
dataset = load_dataset("maritaca-ai/enem")

In [3]:
df = pd.DataFrame(dataset['train'])
df = df.dropna()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            180 non-null    object
 1   exam          180 non-null    object
 2   IU            180 non-null    bool  
 3   ledor         180 non-null    bool  
 4   question      180 non-null    object
 5   alternatives  180 non-null    object
 6   label         180 non-null    object
 7   figures       180 non-null    object
 8   description   180 non-null    object
dtypes: bool(2), object(7)
memory usage: 10.3+ KB


In [5]:
df.alternatives[1]

['risco de choques culturais.',
 'impacto do ensino de história.',
 'importância da luta ambiental.',
 'existência de experiências plurais.',
 'necessidade de capacitação profissional.']

In [6]:
df[['context', 'question']] = df['question'].apply(lambda x: pd.Series(x.rsplit('\n', 1)))

/var/folders/k5/4j1l0hjx1cz26q703gh4y1kh0000gn/T/ipykernel_34584/3025987301.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df[['context', 'question']] = df['question'].apply(lambda x: pd.Series(x.rsplit('\n', 1)))


In [7]:
df['id'] = df['id'].str.extract('(\d+)')
df['id'] = pd.to_numeric(df['id'])

In [8]:
choices = list(ascii_uppercase)
df['options'] = df['alternatives'].apply(lambda x: {choices[i]: valor for i, valor in enumerate(x)})

In [9]:
def extract_label(row):
    alternatives = row['options']
    label = row['label']
    return {label: alternatives.get(label)}

df['correct_answer'] = df.apply(extract_label, axis=1)

In [10]:
def check_image(figures):
    return 'Yes' if figures else 'No'

df['image'] = df['figures'].apply(lambda x: check_image(x))

In [11]:
df['year'] = df['exam']

In [13]:
df = df.drop(columns=['exam', 'alternatives', 'label'])

In [14]:
df

,id,IU,ledor,question,figures,description,context,options,correct_answer,image,year
0,1,False,True,"Nesse poema, a expressão “No man is an island”...",[],[],"No man is an island,\nEntire of itself;\nEvery...","{'A': 'medo da morte.', 'B': 'ideia de conexão...",{'B': 'ideia de conexão.'},No,2023
1,2,False,True,"Ao retratar a trajetória de refugiados, o poem...",[],[],## Things We Carry on the Sea\nWe carry tears ...,"{'A': 'risco de choques culturais.', 'B': 'imp...",{'D': 'existência de experiências plurais.'},No,2023
2,3,True,True,Esse cartaz de campanha sugere que,[https://raw.githubusercontent.com/piresramon/...,[Descrição do cartaz: Foto de parte de um lixã...,[[placeholder]],"{'A': 'os lixões precisam de ampliação.', 'B':...",{'B': 'o desperdício degrada o ambiente.'},Yes,2023
3,4,True,True,Ao retratar o ambiente de trabalho em um escri...,[https://raw.githubusercontent.com/piresramon/...,"[Descrição do cartum: Em um escritório, dois h...",[[placeholder]],"{'A': 'criticar um padrão de vestimenta.', 'B'...",{'B': 'destacar a falta de diversidade.'},Yes,2023
4,5,False,True,"Nesse poema de Tato Laviera, o eu lírico desta...",[],[],(fragmento)\nSpanglish\nTato Laviera\npues est...,"{'A': 'convergência linguístico-cultural.', 'B...",{'A': 'convergência linguístico-cultural.'},No,2023
...,...,...,...,...,...,...,...,...,...,...,...
175,176,True,True,A opção a ser escolhida é aquela em que a medi...,[https://raw.githubusercontent.com/piresramon/...,[Descrição da figura: A figura representa o ma...,O mastro de uma bandeira foi instalado perpend...,"{'A': '22 x sqrt(3)/3.', 'B': '11 x sqrt(2).',...",{'C': '12 x sqrt(2).'},Yes,2023
176,177,True,True,A escala em que essa tela representa as medida...,[https://raw.githubusercontent.com/piresramon/...,"[Descrição do gráfico: Gráfico cartesiano, em ...",Um controlador de voo dispõe de um instrumento...,"{'A': '1 : 5.', 'B': '1 : 11.', 'C': '1 : 55.'...",{'E': '1 : 500 000.'},Yes,2023
177,178,False,True,"De acordo com as informações fornecidas, duran...",[],[],O calendário maia apresenta duas contagens sim...,"{'A': '741', 'B': '1 040', 'C': '1 460', 'D': ...",{'C': '1 460'},No,2023
178,179,True,True,A área da pizza do professor de matemática é m...,[https://raw.githubusercontent.com/piresramon/...,[Descrição da figura: A figura apresenta um tr...,"Sejam a, b e c as medidas dos lados de um triâ...","{'A': '0 grau < alfa < 90 graus', 'B': 'alfa =...",{'C': '90 graus < alfa < 180 graus.'},Yes,2023


In [19]:
result = pd.concat([pd.read_csv('provas_enem.csv'), df], axis=0, join='outer', sort=False, ignore_index=True)

In [20]:
result

,id,year,context,question,options,correct_answer,CE,DS,EK,IC,MR,TC,image,IU,ledor,figures,description
0,1,2013,\nDistribuição espacial atual da população bra...,\nOs mapas representam distintos padrões ...,"{'A': ' fertilização natural dos solos.\n', '...",{'E': ' concentração histórica da urbanização...,No,No,No,Yes,No,No,Yes,NaN,NaN,NaN,NaN
1,2,2013,\nTrata-se de um gigantesco movimento de const...,\nA dinâmica de transformação das cidades...,{'A': ' crescimento da população urbana e aum...,{'A': ' crescimento da população urbana e aum...,No,No,Yes,No,No,No,No,NaN,NaN,NaN,NaN
2,3,2013,"\nNo dia 1º de julho de 2012, a cidade do Rio ...",\nO reconhecimento da paisagem em questão...,{'A': ' presença do corpo artístico local.\n'...,{'E': ' relação sociedade-natureza de caráter...,No,No,Yes,No,No,Yes,No,NaN,NaN,NaN,NaN
3,4,2013,TEXTO I\n\nHá já algum tempo eu me aperc...,\nA exposição e a análise do projeto cartesian...,{'A': ' retomar o método da tradição para edi...,{'B': ' questionar de forma ampla e prof...,No,No,No,No,No,Yes,No,NaN,NaN,NaN,NaN
4,5,2013,"\nDe ponta a ponta, é tudo praia-palma, ...",\nA carta de Pero Vaz de Caminha permit...,{'A': ' Valorizar a catequese a ser real...,{'A': ' Valorizar a catequese a ser real...,No,No,No,No,No,Yes,No,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,176,2023,O mastro de uma bandeira foi instalado perpend...,A opção a ser escolhida é aquela em que a medi...,"{'A': '22 x sqrt(3)/3.', 'B': '11 x sqrt(2).',...",{'C': '12 x sqrt(2).'},NaN,NaN,NaN,NaN,NaN,NaN,Yes,True,True,[https://raw.githubusercontent.com/piresramon/...,[Descrição da figura: A figura representa o ma...
1930,177,2023,Um controlador de voo dispõe de um instrumento...,A escala em que essa tela representa as medida...,"{'A': '1 : 5.', 'B': '1 : 11.', 'C': '1 : 55.'...",{'E': '1 : 500 000.'},NaN,NaN,NaN,NaN,NaN,NaN,Yes,True,True,[https://raw.githubusercontent.com/piresramon/...,"[Descrição do gráfico: Gráfico cartesiano, em ..."
1931,178,2023,O calendário maia apresenta duas contagens sim...,"De acordo com as informações fornecidas, duran...","{'A': '741', 'B': '1 040', 'C': '1 460', 'D': ...",{'C': '1 460'},NaN,NaN,NaN,NaN,NaN,NaN,No,False,True,[],[]
1932,179,2023,"Sejam a, b e c as medidas dos lados de um triâ...",A área da pizza do professor de matemática é m...,"{'A': '0 grau < alfa < 90 graus', 'B': 'alfa =...",{'C': '90 graus < alfa < 180 graus.'},NaN,NaN,NaN,NaN,NaN,NaN,Yes,True,True,[https://raw.githubusercontent.com/piresramon/...,[Descrição da figura: A figura apresenta um tr...


In [21]:
result.to_csv('all_provas_enem.csv', index=False)